In [19]:
from atproto import Client 
from dotenv import dotenv_values 
import asyncio 
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt 
import json
import atproto_client.models as models
import datetime
import dateutil.parser 


dotenv = dotenv_values('.env')

client = Client() 
client.login(dotenv['USERNAME'], dotenv['PASSWORD']);

In [26]:
response = client.app.bsky.feed.search_posts(
    params=models.AppBskyFeedSearchPosts.Params(
        q='#germany',  
        limit=100,
        sort='top'
    )
)

In [29]:
from datetime import datetime, timezone
accounts = {}
for post in response.posts:
    if post.author.handle not in accounts:
        accounts[post.author.handle] = {}
    accounts[post.author.handle]["cumulative posts"] = accounts[post.author.handle].get("cumulative posts", 0) + 1
    accounts[post.author.handle]["cumulative likes"] = accounts[post.author.handle].get("cumulative likes", 0) + post.like_count
    accounts[post.author.handle]["cumulative replys"] = accounts[post.author.handle].get("cumulative replys", 0) + post.reply_count
    accounts[post.author.handle]["cumulative quotes"] = accounts[post.author.handle].get("cumulative quotes", 0) + post.quote_count
    accounts[post.author.handle]["cumulative reposts"] = accounts[post.author.handle].get("cumulative reposts", 0) + post.repost_count
    if accounts[post.author.handle].get("earliest post") is None:
        accounts[post.author.handle]["earliest post"] = datetime.now(timezone.utc)
    accounts[post.author.handle]["earliest post"] = min(accounts[post.author.handle].get("earliest post"), dateutil.parser.isoparse(post.record.created_at))
    accounts[post.author.handle]["account created"] = dateutil.parser.isoparse(post.author.created_at)
    accounts[post.author.handle]["earliest post age"] = accounts[post.author.handle]["earliest post"] - accounts[post.author.handle]["account created"]
    accounts[post.author.handle]["cumulative engagements"] = accounts[post.author.handle].get("cumulative engagements", 0) + post.like_count + post.reply_count + post.quote_count + post.repost_count


In [30]:
accounts

{'kilianschoenberger.bsky.social': {'cumulative posts': 3,
  'cumulative likes': 594,
  'cumulative replys': 8,
  'cumulative quotes': 2,
  'cumulative reposts': 28,
  'earliest post': datetime.datetime(2025, 2, 19, 16, 17, 12, 378000, tzinfo=tzutc()),
  'account created': datetime.datetime(2025, 1, 24, 13, 25, 54, 813000, tzinfo=tzutc()),
  'earliest post age': datetime.timedelta(days=26, seconds=10277, microseconds=565000),
  'cumulative engagements': 632},
 'superchinois801.bsky.social': {'cumulative posts': 1,
  'cumulative likes': 313,
  'cumulative replys': 18,
  'cumulative quotes': 0,
  'cumulative reposts': 18,
  'earliest post': datetime.datetime(2025, 2, 21, 8, 5, 2, 135000, tzinfo=tzutc()),
  'account created': datetime.datetime(2025, 1, 21, 16, 3, 10, 613000, tzinfo=tzutc()),
  'earliest post age': datetime.timedelta(days=30, seconds=57711, microseconds=522000),
  'cumulative engagements': 349},
 'wasbellab102.bsky.social': {'cumulative posts': 1,
  'cumulative likes': 138